<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/john210808/covasim_delta/blob/main/Tools/twitterMsgExtra.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

# Twitter Analysis

## 1. Download twitter 
from  https://www.vicinitas.io/free-tools/download-user-tweets  
    limitation: maximun 3,200 recent tweets 
    
Please save the file in the same fold of notebook. If run in Colab, please first upload the file, or use following download code.

In [1]:
pip install wget

Note: you may need to restart the kernel to use updated packages.


### sample file

In [2]:
import wget
url = 'https://github.com/john210808/covasim_delta/raw/main/Tools/sample.xlsx'
inputName = wget.download(url)
print(inputName)

sample (1).xlsx


### can be used to unzip file

In [3]:
import zipfile

zipfileName = "@BBCPolitics_user_tweets.xlsx.zip"
# with zipfile.ZipFile(zipfileName,"r") as zip_ref:
#     zip_ref.extractall(".")

## 2. Run tools

In [4]:
import pandas as pd
import openpyxl
import os
from openpyxl import load_workbook

baselink = ""

def readXlsx(xlsxName):
    global baselink
    baselink = openpyxl.load_workbook(xlsxName)["tweets"].cell(row=2, column=1).hyperlink.target
    baselink = baselink[:baselink.rfind('/')]
    df = pd.read_excel(xlsxName) 
    df["Time"] = pd.DatetimeIndex(pd.to_datetime(df["UTC"])).tz_convert('Australia/NSW')
    df = df[["Tweet Id", "Text", "Name", "Screen Name", "Time"]]
    df["Tweet Id"] = df["Tweet Id"].apply(str)
    return df


def saveXlsx(df, name, overwrite=False):
    tmpfile = "__temp__.xlsx"
    df["Time"] = df['Time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    df.to_excel(tmpfile, sheet_name='result', index=False)  

    workbook = load_workbook(tmpfile)
    worksheet = workbook.active

    for row in range(2, worksheet.max_row + 1):
        filelocation = worksheet.cell(column=1, row=row)  # this is hyperlink
        c = worksheet.cell(column=1, row=row)
        c.hyperlink = baselink + '/' + c.value

    worksheet.column_dimensions['A'].width = 21
    worksheet.column_dimensions['B'].width = 100
    worksheet.column_dimensions['C'].width = 12
    worksheet.column_dimensions['D'].width = 12
    worksheet.column_dimensions['E'].width = 15
    
    
    if name.rfind(".xlsx") > 0:
        name = name[:name.rfind(".xlsx")]
    
    if os.path.isfile(name + ".xlsx") and not overwrite:
        files = [f for f in os.listdir(".") if os.path.isfile(f) and f.endswith(".xlsx") and f.startswith(name)]
        name = '%s%03d' % (name, len(files)) #datetime.now().strftime("%m%d%H%M%S")
    
    workbook.save(name + ".xlsx")
    os.remove(tmpfile)
    
def process(xlsxName, outputName, start_time, end_time, words, authors = []):
    df = readXlsx(xlsxName)

    # conditions
    cond_time = (df['Time'] >= start_time) & (df['Time'] <= end_time)
    df = df.loc[cond_time]
    
    text = df["Text"].str
    isfirst = False

    for i in range(len(words)):
        if i == 0:
            cond_text = text.contains(words[i]) 
        else:
            cond_text = cond_text | text.contains(words[i]) 

#     name = df["Name"].str
#     for i in range(len(authors)):
#         if i == 0:
#             cond_author = name.contains(authors[i]) 
#         else:
#             cond_author = cond_author | name.contains(authors[i]) 

    df = df.loc[cond_text]
    saveXlsx(df, "output", True)
    print("totoal number: %d" % df.shape[0])
    
    df.drop(df.index, inplace=True) # reset df

## 3. Change condition and export excels

In [5]:
start_time = pd.Timestamp("2021-06-10 00:00:00", tz='Australia/NSW')
end_time =  pd.Timestamp("2021-08-20 23:59:59", tz='Australia/NSW')
words = ["Hi", "sewage"]  # find texts that contain these words
#authors = ["NSW Health"]  # author

inputName = 'sample.xlsx' # the downloaded excel file
outName = "output.xlsx"   # the output

process(inputName, outName, start_time, end_time, words);

totoal number: 18
